# Bash file

In [2]:
def bash(string,name,queue):
  script = "%s.sh"%name
  with open(script, "w") as text_file:
    text_file.write(string)
  !chmod +x $script
  !qsub -q $queue $script
  #!rm $script
  
ws = '/beegfs/work/workspace/ws/ho_rettu74-genomes-0/jupyterlab/'

#Create list of file indexes to access it later
import os
bams = [bam.split("_")[0] for bam in os.listdir("Raw_data")]
print(bams)

['J21', 'E2', 'B98', 'B2']


# Unzip and move files

In [ ]:
tarfile='X204SC21100035-Z01-F002_01.tar'
import tarfile
tar = tarfile.open('X204SC21100035-Z01-F002_01.tar')
tar.extractall(path='X204SC21100035-Z01-F002_01.tar')
tar.close()

In [2]:
# move files to Raw_reads
import os
for di in os.listdir('X204SC21100035-Z01-F002_01/raw_data/'):
  if di != 'Rawdata_Readme.pdf':
    !mv X204SC21100035-Z01-F002_01/raw_data/$di/*subreads.bam Raw_data/

# Transform bam to fastq

In [7]:
##Samtools = .fastq
string = '''
#PBS -l nodes=1:ppn=2
#PBS -l walltime=24:00:00
#PBS -l mem=64gb
#PBS -S /bin/bash
# Load the module
module load bio/samtools/1.9
# Run
workdir=%s
name=%s
cd $workdir
samtools fastq -N -O Raw_data/${name}_*.subreads.bam > fq_reads/${name}.fastq
'''
!mkdir "fq_reads"
for bam in bams:
    if bam == bams[0]:
        continue
    out = "fq_reads/%s.fastq" %bam
    if os.path.exists(out): continue
    bash(string%(ws,bam),'%s_samtools'%bam,'short')

mkdir: cannot create directory ‘fq_reads’: File exists
10684663
10684664
10684665


# Step 1: Generating assemblies

In [11]:
##Filtlong = reads.fastq
string = '''
#PBS -l nodes=1:ppn=2
#PBS -l walltime=48:00:00
#PBS -l mem=32gb
#PBS -S /bin/bash
# Activate conda enviroment
source /home/ho/ho_genetics/ho_rettu74/miniconda3/etc/profile.d/conda.sh
conda activate filtlong
workdir=%s
name=%s
cd $workdir
#filtlong
filtlong --min_length 1000 --keep_percent 95 fq_reads/${name}.fastq > filtlong/${name}_reads.fastq
'''
!mkdir filtlong
for bam in bams:
    bash(string%(ws,bam),'%s_filtlong'%bam,'short')

mkdir: cannot create directory ‘filtlong’: File exists
10684696
10684697
10684698
10684699


In [17]:
##Subsampling reads (trycycler) = read_subsets
string = '''
#PBS -l nodes=1:ppn=10
#PBS -l walltime=24:00:00
#PBS -l mem=250gb
#PBS -S /bin/bash
source /home/ho/ho_genetics/ho_rettu74/miniconda3/etc/profile.d/conda.sh
conda activate trycycler
workdir3=%s
name=%s
size=%s
cd $workdir3
trycycler subsample --reads filtlong/${name}_reads.fastq --genome_size $size --out_dir subsets/$name --threads 10
conda deactivate
'''
sizes= {'J21':"2.5m", 'E2':"2.5m", 'B98': "2.5m", 'B2': "2.5m"}
for bam in bams:
    !mkdir -p subsets/$bam
    bash(string%(ws,bam,sizes[bam]),'%s_subsets'%bam,'smp')

10685021
10685022
10685023
10685024


In [21]:
##Assemblies (flye)
string = '''
#PBS -l nodes=1:ppn=20
#PBS -l walltime=24:00:00
#PBS -l mem=250gb
#PBS -q smp
#PBS -S /bin/bash
source /home/ho/ho_genetics/ho_rettu74/miniconda3/etc/profile.d/conda.sh
conda activate flye
workdir4=%s
name=%s
cd $workdir4
flye --pacbio-raw subsets/$name/sample_01.fastq --threads 20 --out-dir assemblies/$name/assembly_01 && cp assemblies/$name/assembly_01/assembly.fasta assemblies/${name}_flye_01.fasta && rm -r assembly_01
flye --pacbio-raw subsets/$name/sample_02.fastq --threads 20 --out-dir assemblies/$name/assembly_02 && cp assemblies/$name/assembly_02/assembly.fasta assemblies/${name}_flye_02.fasta && rm -r assembly_02
flye --pacbio-raw subsets/$name/sample_03.fastq --threads 20 --out-dir assemblies/$name/assembly_03 && cp assemblies/$name/assembly_01/assembly.fasta assemblies/${name}_flye_03.fasta && rm -r assembly_03
flye --pacbio-raw subsets/$name/sample_04.fastq --threads 20 --out-dir assemblies/$name/assembly_14 && cp assemblies/$name/assembly_01/assembly.fasta assemblies/${name}_flye_04.fasta && rm -r assembly_04
flye --pacbio-raw subsets/$name/sample_05.fastq --threads 20 --out-dir assemblies/$name/assembly_05 && cp assemblies/$name/assembly_01/assembly.fasta assemblies/${name}_flye_05.fasta && rm -r assembly_05
flye --pacbio-raw subsets/$name/sample_06.fastq --threads 20 --out-dir assemblies/$name/assembly_06 && cp assemblies/$name/assembly_01/assembly.fasta assemblies/${name}_flye_06.fasta && rm -r assembly_06
flye --pacbio-raw subsets/$name/sample_07.fastq --threads 20 --out-dir assemblies/$name/assembly_07 && cp assemblies/$name/assembly_01/assembly.fasta assemblies/${name}_flye_07.fasta && rm -r assembly_07
flye --pacbio-raw subsets/$name/sample_08.fastq --threads 20 --out-dir assemblies/$name/assembly_08 && cp assemblies/$name/assembly_01/assembly.fasta assemblies/${name}_flye_08.fasta && rm -r assembly_08
flye --pacbio-raw subsets/$name/sample_09.fastq --threads 20 --out-dir assemblies/$name/assembly_09 && cp assemblies/$name/assembly_01/assembly.fasta assemblies/${name}_flye_09.fasta && rm -r assembly_09
flye --pacbio-raw subsets/$name/sample_10.fastq --threads 20 --out-dir assemblies/$name/assembly_10 && cp assemblies/$name/assembly_01/assembly.fasta assemblies/${name}_flye_10.fasta && rm -r assembly_10
flye --pacbio-raw subsets/$name/sample_11.fastq --threads 20 --out-dir assemblies/$name/assembly_11 && cp assemblies/$name/assembly_01/assembly.fasta assemblies/${name}_flye_11.fasta && rm -r assembly_11
flye --pacbio-raw subsets/$name/sample_12.fastq --threads 20 --out-dir assemblies/$name/assembly_12 && cp assemblies/$name/assembly_01/assembly.fasta assemblies/${name}_flye_12.fasta && rm -r assembly_12
conda deactivate
'''
for bam in bams:
    !mkdir -p assemblies/$bam
    bash(string%(ws,bam),'%s_assemblies'%bam,'smp')

10685104
10685105
10685106
10685107


# Step 2: Clustering contigs

In [21]:
##Running Trycycler cluster
string = '''
#PBS -l nodes=1:ppn=10
#PBS -l walltime=24:00:00
#PBS -l mem=250gb
#PBS -S /bin/bash
source /home/ho/ho_genetics/ho_rettu74/miniconda3/etc/profile.d/conda.sh
conda activate trycycler

workdir=%s
name=%s
cd $workdir

trycycler cluster --assemblies assemblies/${name}_flye_*.fasta --reads filtlong/${name}_reads.fastq --out_dir clusters/$name --threads 10
conda deactivate
'''
for bam in bams:
    !mkdir -p  clusters/$bam 
    # !mv /clusters/%s_cluster/contigs.* /clusters/%s_cluster/%s_contigs.*
    bash(string%(ws,bam),'%s_clusters'%bam,'smp')

10707154
10707155
10707156
10707157


In [ ]:
# Step 3: Reconciling contigs

In [90]:
##Running Trycycler reconcile
#using the best cluster
##have to be run separated to each cluster
string = '''
#PBS -l nodes=1:ppn=20
#PBS -l walltime=24:00:00
#PBS -l mem=250gb
#PBS -q smp
#PBS -S /bin/bash
source /home/ho/ho_genetics/ho_rettu74/miniconda3/etc/profile.d/conda.sh
conda activate trycycler
workdir=%s
name=%s
cd $workdir
trycycler reconcile --reads filtlong/${name}_reads.fastq --cluster_dir clusters/${name}/cluster_001
conda deactivate
'''
#for bam in bams:
    #!mkdir -p  reconcile/$bam 
bash(string%(ws,bam),'%s_reconcile'%bam,'smp')

10707913


# Step 4: Multiple sequence alignment

In [117]:
##Running Trycycler msa
string = '''
#PBS -l nodes=1:ppn=20
#PBS -l walltime=24:00:00
#PBS -l mem=250gb
#PBS -q smp
#PBS -S /bin/bash
source /home/ho/ho_genetics/ho_rettu74/miniconda3/etc/profile.d/conda.sh
conda activate trycycler
workdir=%s
name=%s
cd $workdir
trycycler msa --cluster_dir clusters/${name}/cluster_001
conda deactivate
'''
for bam in bams:
    #!mkdir -p  reconcile/$bam 
    bash(string%(ws,bam),'%s_msa'%bam,'smp')

10707921
10707922
10707923
10707924


# Step 5: Partitioning reads

In [3]:
##Running Trycycler partition
string = '''
#PBS -l nodes=1:ppn=20
#PBS -l walltime=24:00:00
#PBS -l mem=250gb
#PBS -q smp
#PBS -S /bin/bash
source /home/ho/ho_genetics/ho_rettu74/miniconda3/etc/profile.d/conda.sh
conda activate trycycler
workdir=%s
name=%s
cd $workdir
trycycler partition --reads filtlong/${name}_reads.fastq --cluster_dir clusters/${name}/cluster_001
conda deactivate
'''
for bam in bams:
    #!mkdir -p  reconcile/$bam 
    bash(string%(ws,bam),'%s_partition'%bam,'smp')

10708041
10708042
10708043
10708044


# Step 6: Generating a consensus

In [17]:
##Running Trycycler consensus
string = '''
#PBS -l nodes=1:ppn=20
#PBS -l walltime=24:00:00
#PBS -l mem=250gb
#PBS -q smp
#PBS -S /bin/bash
source /home/ho/ho_genetics/ho_rettu74/miniconda3/etc/profile.d/conda.sh
conda activate trycycler
workdir=%s
name=%s
cd $workdir
trycycler consensus --cluster_dir clusters/${name}/cluster_001
conda deactivate
'''
for bam in bams:
    #!mkdir -p  reconcile/$bam 
    bash(string%(ws,bam),'%s_consensus'%bam,'smp')

10708046
10708047
10708048
10708049


# Step 7: Polishing after Trycycler

In [ ]:
##Running 
string = '''
#PBS -l nodes=1:ppn=20
#PBS -l walltime=24:00:00
#PBS -l mem=250gb
#PBS -q smp
#PBS -S /bin/bash
source /home/ho/ho_genetics/ho_rettu74/miniconda3/etc/profile.d/conda.sh
conda activate trycycler
workdir=%s
name=%s
cd $workdir

